In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date, time

import warnings
warnings.filterwarnings("ignore")

In [2]:
from datetime import datetime, date, time
now = datetime.now()

# Step 1 loading the original dataset and the scraped 

In [3]:
#Pb1 This data wrangling cleaning 
Project_A=pd.read_csv('listings_air.csv')
project_b=pd.read_csv('project_b.csv')

In [6]:
project_b.head(3)

,Unnamed: 0,available,available_for_checkin,date,max_nights,min_nights,price,ID,local_adjusted_price,type,date_scraped
0,0,False,False,2018-10-01,1125,3,"{'date': '2018-10-01', 'local_adjusted_price':...",63864,128,custom,2018-12-10
1,1,False,False,2018-10-02,1125,3,"{'date': '2018-10-02', 'local_adjusted_price':...",63864,128,custom,2018-12-10
2,2,False,False,2018-10-03,1125,3,"{'date': '2018-10-03', 'local_adjusted_price':...",63864,128,custom,2018-12-10


# Step 1, Removing unwanted data from the scraped dataset

### Some Columns are duplications or have null values. 

In [7]:
# Dropping the columns with NA's 
project_b.dropna(inplace=True)

In [8]:
# Droping unneeded columns
project_b.drop(labels=['price','available_for_checkin'],axis=1,inplace=True)

In [10]:
# Dropping duplicate rows 
project_b.drop_duplicates(inplace=True)

# Step 2 Renaming Columns in the scraped dataset 

### Important to rename features so they can be clearly understood 

In [11]:
# Adapting the list into a usable structer 
project_b.rename(columns={"local_adjusted_price": "av_price",'ID':'id','date':'date3',
                          'available':'ave_availble','max_nights':'ave_max_nights',
                          'min_nights':'ave_min_nights'},inplace=True)

# Step 3 Correcting data types

### Through the scraping process some data may have been saved not in the correct format. This is to update this mistakes. 

In [12]:
project_b['date3'] = pd.to_datetime(project_b.date3)

project_b['id']=project_b['id'].apply(str)

# Step 4 Grouping the scraped data 

### Grouping the data always to get an average of when a listing is occupied.

In [14]:
# Grouping the df by "id" to get average values
fgh=project_b.groupby('id').mean()


# Project_A.index.name = None
fgh.index.name = None

# Step 5 Joining the original database and the scraped data

### This then gives a master a database to preform investigation on. 

In [15]:
Project_B_Final=Project_A.join(fgh,how='left',)

In [16]:
fgh.head(3)

,Unnamed: 0,ave_availble,ave_max_nights,ave_min_nights,av_price
147244,17.0,0.257143,1125.0,4.100000,154.871429
194664,17.0,0.157143,1125.0,4.385714,131.228571
207784,17.0,0.157143,1125.0,3.942857,147.400000


# Step 6 Saving the cleaned scraped database

### This is just a precaution ahead of cleaning the new master database. 

In [17]:
today = now.strftime("%Y-%m-%d")

project_b.to_csv('project_b_Scrape_decemebr.csv')

In [19]:
project_b.head(3)

,Unnamed: 0,ave_availble,date3,ave_max_nights,ave_min_nights,id,av_price,type,date_scraped
0,0,False,2018-10-01,1125,3,63864,128,custom,2018-12-10
1,1,False,2018-10-02,1125,3,63864,128,custom,2018-12-10
2,2,False,2018-10-03,1125,3,63864,128,custom,2018-12-10


# Step 7 Clean the master database

In [20]:
# Loading the master database
Project_B_FinalV2=pd.read_csv("project_b_restructr.csv")

In [21]:
Project_B_FinalV2.rename(columns={'Unnamed: 0': "id"},inplace=True)

In [22]:
#Project_B_FinalV2['acc_ave_month_occ'].mean()

In [23]:
# Creating a column of just the average 
Project_B_FinalV2['average']=Project_B_FinalV2['acc_ave_month_occ'].mean()

In [24]:
# Fixing so the occ pancy level of inside Airbnb doesnt go over 21 days (70%) per month
Project_B_FinalV2.Insi_ave_month_occ[Project_B_FinalV2['Insi_ave_month_occ']>21]=21

In [25]:
# Checking the final database size 
Project_B_FinalV2.shape

(5280, 40)

In [26]:
# unsure if I need this ....
host_listings_count = 20
maximum_nights =30
instant_bookable = "t"

# #Pb3
# # Filtering by host listing counts
mask_biz=Project_B_FinalV2.host_listings_count>host_listings_count
listingv2=Project_B_FinalV2[mask_biz]

# # listingv2.shape

# # Filtering by maxiumum nights
m2=listingv2.ave_max_nights>=maximum_nights
listingv3=listingv2[m2]

# ## This may need to be switched back after, below
# # # Filtering by wether if its instant bookable
# # m3=listingv3['instant_bookable']== instant_bookable 
# # Project_B_FinalV2=listingv3[m3]

Project_B_FinalV2=listingv3

# Step 8, Making the target catagorical

### This is so when can try classification modeling if time is available 

In [27]:
# This categorizes the occpancy levels so classifacation can be tested 
Project_B_FinalV2['high_low']=pd.qcut(Project_B_FinalV2['acc_ave_month_occ'],215,
                                      labels=None,retbins=False, duplicates='drop').astype(str)

In [28]:
# taking out Inside Airbnb results for regression modeling, the average and the high_low 
# to see not effect the model. 
average=Project_B_FinalV2.pop('average')
high_low=Project_B_FinalV2.pop('high_low')

# Step 9, Dummifing the data, 

### This turns all catagorical data into binary 

In [30]:
# Dummifies all the objects
Project_B_Dummied2=pd.get_dummies(Project_B_FinalV2,drop_first=True)

#This is trying to dummify cataorgical columns)(that appear to be numerical)
Project_B_Dummied3=pd.get_dummies(Project_B_Dummied2,drop_first=True,columns=['review_scores_accuracy',
                                                                              'review_scores_cleanliness',
                                                                              'review_scores_checkin',
                                                                              'review_scores_communication',
                                                                              'review_scores_location',
                                                                              'review_scores_value'])

Project_B_Dummied3.head(3)

,id,host_response_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,guests_included,extra_people,...,review_scores_location_8.0,review_scores_location_9.0,review_scores_location_10.0,review_scores_value_4.0,review_scores_value_5.0,review_scores_value_6.0,review_scores_value_7.0,review_scores_value_8.0,review_scores_value_9.0,review_scores_value_10.0
0,24903,100.0,53.0,53.0,5,2.0,2.0,2.0,3,20.0,...,0,0,0,0,0,0,1,0,0,0
6,63864,100.0,53.0,53.0,3,1.0,1.0,1.0,2,35.0,...,0,1,0,0,0,0,0,1,0,0
17,207784,100.0,53.0,53.0,5,2.0,2.0,2.0,3,20.0,...,0,1,0,0,0,0,0,0,1,0


# Step 10,  Saving the final cleaned and dummified dataset

In [31]:
Project_B_Dummied3.shape

(1827, 114)

In [32]:
Project_B_Dummied4=Project_B_Dummied3.copy()

# Step 6a, Saving the dummified data
today = now.strftime("%Y-%m-%d")
Project_B_Dummied4.to_csv('project_B2_dummied.csv')
